In [2]:
%%capture
!pip install datasets

In [3]:
%%capture
!pip install transformers

In [4]:
# from google.colab import drive
# drive.mount('/content/drive')

In [5]:
train_data_path = '/content/drive/MyDrive/week 8/train-v2.0-translated.json'

In [6]:
dev_data_path = '/content/drive/MyDrive/week 8/dev-v2.0-translated.json'

In [7]:
import json
from datasets import DatasetDict, Dataset

In [8]:
with open(train_data_path, "r", encoding="utf-8") as f:
    train_data = json.load(f)[:1303]

with open(dev_data_path, "r", encoding="utf-8") as f:
    dev_data = json.load(f)[:1000]

In [9]:
def find_answer_start(context, answer):
    """
    Tìm vị trí bắt đầu của câu trả lời trong ngữ cảnh.

    Parameters:
    - context (str): Văn bản ngữ cảnh.
    - answer (str): Câu trả lời cần tìm.

    Returns:
    - int: Vị trí bắt đầu của câu trả lời trong ngữ cảnh hoặc -1 nếu không tìm thấy.
    """
    start_index = context.find(answer)
    return start_index

# Ví dụ
context = "Beyoncé Giselle Knowles-Carter (/ b i gì ɒ n s eɪ / bee-YON-say) (sinh ngày 04 tháng 9 1981) là một ca sĩ, nhạc sĩ, nhà sản xuất thu âm và nữ diễn viên người Mỹ."
answer = "nhà sản xuất thu âm"

# Tìm kiếm answer_start
answer_start = find_answer_start(context, answer)
print("Answer start index:", answer_start)

Answer start index: 116


In [10]:
# Hàm tìm kiếm answer_start
def find_answer_start(context, answer):
    return context.find(answer)

# Chuyển đổi dữ liệu thành dạng dict
def transform_data(data):
    transformed = []
    for item in data:
        context = item[0]
        question = item[1]
        answer_text = item[2]

        # Tạo answers chỉ với text
        answers = {'text': [answer_text]}  # Loại bỏ answer_start

        transformed.append({
            'context': context,
            'question': question,
            'answers': answers
        })
    return transformed

# Chuyển đổi dữ liệu train và dev
train_transformed = transform_data(train_data)
dev_transformed = transform_data(dev_data)

# Tạo Dataset từ dữ liệu đã chuyển đổi
train_dataset = Dataset.from_dict({
    'context': [item['context'] for item in train_transformed],
    'question': [item['question'] for item in train_transformed],
    'answers': [item['answers'] for item in train_transformed]
})

dev_dataset = Dataset.from_dict({
    'context': [item['context'] for item in dev_transformed],
    'question': [item['question'] for item in dev_transformed],
    'answers': [item['answers'] for item in dev_transformed]
})

# Tạo DatasetDict
raw_datasets = DatasetDict({
    'train': train_dataset,
    'validation': dev_dataset
})

In [11]:
print(raw_datasets)

DatasetDict({
    train: Dataset({
        features: ['context', 'question', 'answers'],
        num_rows: 1303
    })
    validation: Dataset({
        features: ['context', 'question', 'answers'],
        num_rows: 1000
    })
})


In [12]:
print("context : ", raw_datasets['train'][0]['context'])
print("question : ", raw_datasets['train'][0]['question'])
print("answers : ", raw_datasets['train'][0]['answers'])

context :  Beyoncé Giselle Knowles-Carter (/ b i gì ɒ n s eɪ / bee-YON-say) (sinh ngày 04 tháng 9 1981) là một ca sĩ, nhạc sĩ, nhà sản xuất thu âm và nữ diễn viên người Mỹ. Sinh ra và lớn lên ở Houston, Texas, cô đã biểu diễn trong các cuộc thi ca hát và nhảy múa khác nhau khi còn nhỏ, và nổi tiếng vào cuối những năm 1990 với tư cách là ca sĩ chính của nhóm nhạc nữ R & B Destiny's Child. Được quản lý bởi cha cô, Mathew Knowles, nhóm đã trở thành một trong những nhóm nhạc nữ bán chạy nhất thế giới mọi thời đại. Sự gián đoạn của họ đã chứng kiến việc phát hành album đầu tay của Beyoncé, Dangerously in Love (2003), giúp cô trở thành một nghệ sĩ solo trên toàn thế giới, giành được năm giải Grammy và có đĩa đơn quán quân Billboard Hot 100 "Crazy in Love" và "Baby Boy".
question :  Beyonce bắt đầu nổi tiếng từ khi nào?
answers :  {'text': ['Vào cuối những năm 1990']}


In [13]:
raw_datasets['train'][0]

{'context': 'Beyoncé Giselle Knowles-Carter (/ b i gì ɒ n s eɪ / bee-YON-say) (sinh ngày 04 tháng 9 1981) là một ca sĩ, nhạc sĩ, nhà sản xuất thu âm và nữ diễn viên người Mỹ. Sinh ra và lớn lên ở Houston, Texas, cô đã biểu diễn trong các cuộc thi ca hát và nhảy múa khác nhau khi còn nhỏ, và nổi tiếng vào cuối những năm 1990 với tư cách là ca sĩ chính của nhóm nhạc nữ R & B Destiny\'s Child. Được quản lý bởi cha cô, Mathew Knowles, nhóm đã trở thành một trong những nhóm nhạc nữ bán chạy nhất thế giới mọi thời đại. Sự gián đoạn của họ đã chứng kiến việc phát hành album đầu tay của Beyoncé, Dangerously in Love (2003), giúp cô trở thành một nghệ sĩ solo trên toàn thế giới, giành được năm giải Grammy và có đĩa đơn quán quân Billboard Hot 100 "Crazy in Love" và "Baby Boy".',
 'question': 'Beyonce bắt đầu nổi tiếng từ khi nào?',
 'answers': {'text': ['Vào cuối những năm 1990']}}

In [14]:
raw_datasets['train'][0]['answers']['text'][0]

'Vào cuối những năm 1990'

In [15]:
raw_datasets["train"].filter(lambda x: len(x["answers"]["text"]) != 1)

Filter:   0%|          | 0/1303 [00:00<?, ? examples/s]

Dataset({
    features: ['context', 'question', 'answers'],
    num_rows: 0
})

In [16]:
print(raw_datasets["validation"][2]["answers"])

{'text': ['Đan Mạch, Iceland và Na Uy']}


In [17]:
print(raw_datasets["validation"][2]["context"])
print(raw_datasets["validation"][2]["question"])

Người Norman (Norman: Nourmands; French: Normands; Latin: Normanni) là những người trong thế kỷ 10 và 11 đã đặt tên cho Normandy, một khu vực ở Pháp. Họ là hậu duệ của những kẻ cướp biển và cướp biển Norse ("Norman" đến từ "Norseman") từ Đan Mạch, Iceland và Na Uy, dưới sự lãnh đạo của Rollo, đã đồng ý thề trung thành với Vua Charles III của Tây Francia. Qua nhiều thế hệ đồng hoá và trộn lẫn với dân số Frank và La Mã-Gaulish bản địa, con cháu của họ sẽ dần dần hợp nhất với các nền văn hoá dựa trên Carolingian của Tây Francia. Bản sắc văn hoá và dân tộc riêng biệt của người Norman xuất hiện ban đầu vào nửa đầu thế kỷ 10, và nó tiếp tục phát triển trong các thế kỷ tiếp theo.
Người Na Uy có nguồn gốc từ quốc gia nào?


=> Câu trả lời là 1 trong 3 text ở trên

# 2. Preprocessing the training data

In [18]:
from transformers import AutoTokenizer

model_checkpoint = "bert-base-cased"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [19]:
tokenizer.is_fast

True

Thêm các token [CLS] và [SEP] để mô hình hiểu.

[CLS]: Classification.
[SEP]: Token phân cách câu hỏi và ngữ cảnh, giữa các đoạn văn bản khác nhau.
[UNK]: Unknown

In [20]:
context = raw_datasets["train"][0]["context"]
question = raw_datasets["train"][0]["question"]

inputs = tokenizer(question, context)
tokenizer.decode(inputs["input_ids"])

'[CLS] Beyonce bắt đầu nổi tiếng từ khi nào? [SEP] Beyoncé Giselle Knowles - Carter ( / b i gì [UNK] n s eɪ / bee - YON - say ) ( sinh ngày 04 tháng 9 1981 ) là một ca sĩ, nhạc sĩ, nhà sản xuất thu âm và nữ diễn viên người Mỹ. Sinh ra và lớn lên [UNK] Houston, Texas, cô đã biểu diễn trong các cuộc thi ca hát và nhảy múa khác nhau khi còn [UNK], và nổi tiếng vào cuối những năm 1990 với tư cách là ca sĩ chính của nhóm nhạc nữ R & B Destiny\'s Child. Được quản lý [UNK] cha cô, Mathew Knowles, nhóm đã [UNK] thành một trong những nhóm nhạc nữ bán chạy nhất thế giới mọi thời đại. Sự gián đoạn của họ đã chứng kiến việc phát hành album đầu tay của Beyoncé, Dangerously in Love ( 2003 ), giúp cô [UNK] thành một nghệ sĩ solo trên toàn thế giới, giành được năm giải Grammy và có đĩa đơn quán quân Billboard Hot 100 " Crazy in Love " và " Baby Boy ". [SEP]'

In [21]:
count = 0;
for i in inputs['input_ids']: # 1D
  if i==102:
    count = count + 1;
print(count);

2


In [22]:
obj = tokenizer(
    question,
    context,
    max_length=100,
    truncation=True, # by default trucation of second one i.e. context
    return_overflowing_tokens=True,
)

for ids in obj["input_ids"]:
    print(tokenizer.decode(ids))

print(len(obj['input_ids']))
print(len(obj['input_ids'][0]))
print(len(obj['input_ids'][1]))

[CLS] Beyonce bắt đầu nổi tiếng từ khi nào? [SEP] Beyoncé Giselle Knowles - Carter ( / b i gì [UNK] n s eɪ / bee - YON - say ) ( sinh ngày 04 tháng 9 1981 ) là một ca sĩ, nhạc sĩ, nhà sản xuất thu âm và nữ di [SEP]
[CLS] Beyonce bắt đầu nổi tiếng từ khi nào? [SEP]ễn viên người Mỹ. Sinh ra và lớn lên [UNK] Houston, Texas, cô đã biểu diễn trong các cuộc thi ca hát và nhảy múa khác nhau khi c [SEP]
[CLS] Beyonce bắt đầu nổi tiếng từ khi nào? [SEP]òn [UNK], và nổi tiếng vào cuối những năm 1990 với tư cách là ca sĩ chính của nhóm nhạc nữ R & B Destiny's Child. Được quản l [SEP]
[CLS] Beyonce bắt đầu nổi tiếng từ khi nào? [SEP]ý [UNK] cha cô, Mathew Knowles, nhóm đã [UNK] thành một trong những nhóm nhạc nữ bán chạy nhất thế giới mọi thời đại. Sự gián đoạ [SEP]
[CLS] Beyonce bắt đầu nổi tiếng từ khi nào? [SEP]n của họ đã chứng kiến việc phát hành album đầu tay của Beyoncé, Dangerously in Love ( 2003 ), giúp cô [UNK] thành một nghệ sĩ solo trên toàn thế [SEP]
[CLS] Beyonce bắt đầu nổi tiếng từ

In [23]:
obj1 = tokenizer(
    question,
    context,
    max_length=100,
    truncation=True,
    stride = 50,
    return_overflowing_tokens=True,
)

for ids in obj1["input_ids"]:
    print(tokenizer.decode(ids))

print(len(obj1['input_ids']))
print(len(obj1['input_ids'][0]))
print(len(obj1['input_ids'][1]))
print(len(obj1['input_ids'][2]))
print(len(obj1['input_ids'][3]))

[CLS] Beyonce bắt đầu nổi tiếng từ khi nào? [SEP] Beyoncé Giselle Knowles - Carter ( / b i gì [UNK] n s eɪ / bee - YON - say ) ( sinh ngày 04 tháng 9 1981 ) là một ca sĩ, nhạc sĩ, nhà sản xuất thu âm và nữ di [SEP]
[CLS] Beyonce bắt đầu nổi tiếng từ khi nào? [SEP] say ) ( sinh ngày 04 tháng 9 1981 ) là một ca sĩ, nhạc sĩ, nhà sản xuất thu âm và nữ diễn viên người Mỹ. Sinh ra và lớn l [SEP]
[CLS] Beyonce bắt đầu nổi tiếng từ khi nào? [SEP], nhạc sĩ, nhà sản xuất thu âm và nữ diễn viên người Mỹ. Sinh ra và lớn lên [UNK] Houston, Texas, cô đã biểu diễn trong các cu [SEP]
[CLS] Beyonce bắt đầu nổi tiếng từ khi nào? [SEP]à nữ diễn viên người Mỹ. Sinh ra và lớn lên [UNK] Houston, Texas, cô đã biểu diễn trong các cuộc thi ca hát và nhảy múa khác nha [SEP]
[CLS] Beyonce bắt đầu nổi tiếng từ khi nào? [SEP] lớn lên [UNK] Houston, Texas, cô đã biểu diễn trong các cuộc thi ca hát và nhảy múa khác nhau khi còn [UNK], và nổi tiếng vào cuối [SEP]
[CLS] Beyonce bắt đầu nổi tiếng từ khi nào? [SEP] các 

In [24]:
# to observe strides of 50

obj2 = tokenizer("Architecturally, the school has a Catholic character. Atop the Main Building's gold dome is a golden statue of the Virgin Mary. Immediately in front of ")
print(len(obj2['input_ids']))
obj3 = tokenizer("[CLS] To whom did the Virgin Mary allegedly appear in 1858 in Lourdes France? [SEP] Architecturally, the school has a Catholic character. Atop the Main Building's gold dome is a golden statue of the Virgin Mary. Immediately in front of")
print(len(obj3['input_ids']))

34
51


In [25]:
obj4 = tokenizer(context,max_length=100,truncation=True,return_overflowing_tokens=True)
for ids in obj4["input_ids"]:
    print(tokenizer.decode(ids))

[CLS] Beyoncé Giselle Knowles - Carter ( / b i gì [UNK] n s eɪ / bee - YON - say ) ( sinh ngày 04 tháng 9 1981 ) là một ca sĩ, nhạc sĩ, nhà sản xuất thu âm và nữ diễn viên người Mỹ. Sinh ra và lớn lên [SEP]
[CLS] [UNK] Houston, Texas, cô đã biểu diễn trong các cuộc thi ca hát và nhảy múa khác nhau khi còn [UNK], và nổi tiếng vào cuối những năm 1990 với tư cách là ca sĩ chính của nhóm [SEP]
[CLS] nhạc nữ R & B Destiny's Child. Được quản lý [UNK] cha cô, Mathew Knowles, nhóm đã [UNK] thành một trong những nhóm nhạc nữ bán chạy nhất thế giới mọi thời đại. Sự gián đoạn c [SEP]
[CLS]ủa họ đã chứng kiến việc phát hành album đầu tay của Beyoncé, Dangerously in Love ( 2003 ), giúp cô [UNK] thành một nghệ sĩ solo trên toàn thế giới, giành được năm giải Grammy và có đĩ [SEP]
[CLS]a đơn quán quân Billboard Hot 100 " Crazy in Love " và " Baby Boy ". [SEP]


In [26]:
inputs = tokenizer(
    question,
    context,
    max_length=100,
    truncation="only_second",
    stride=50,
    return_overflowing_tokens=True,
)

for ids in inputs["input_ids"]:
    print(tokenizer.decode(ids))

[CLS] Beyonce bắt đầu nổi tiếng từ khi nào? [SEP] Beyoncé Giselle Knowles - Carter ( / b i gì [UNK] n s eɪ / bee - YON - say ) ( sinh ngày 04 tháng 9 1981 ) là một ca sĩ, nhạc sĩ, nhà sản xuất thu âm và nữ di [SEP]
[CLS] Beyonce bắt đầu nổi tiếng từ khi nào? [SEP] say ) ( sinh ngày 04 tháng 9 1981 ) là một ca sĩ, nhạc sĩ, nhà sản xuất thu âm và nữ diễn viên người Mỹ. Sinh ra và lớn l [SEP]
[CLS] Beyonce bắt đầu nổi tiếng từ khi nào? [SEP], nhạc sĩ, nhà sản xuất thu âm và nữ diễn viên người Mỹ. Sinh ra và lớn lên [UNK] Houston, Texas, cô đã biểu diễn trong các cu [SEP]
[CLS] Beyonce bắt đầu nổi tiếng từ khi nào? [SEP]à nữ diễn viên người Mỹ. Sinh ra và lớn lên [UNK] Houston, Texas, cô đã biểu diễn trong các cuộc thi ca hát và nhảy múa khác nha [SEP]
[CLS] Beyonce bắt đầu nổi tiếng từ khi nào? [SEP] lớn lên [UNK] Houston, Texas, cô đã biểu diễn trong các cuộc thi ca hát và nhảy múa khác nhau khi còn [UNK], và nổi tiếng vào cuối [SEP]
[CLS] Beyonce bắt đầu nổi tiếng từ khi nào? [SEP] các 

In [27]:
inputs = tokenizer(
    question,
    context,
    max_length=100,
    truncation="only_second",
    stride=50,
    return_overflowing_tokens=True,
    return_offsets_mapping=True,
)
inputs.keys()

dict_keys(['input_ids', 'token_type_ids', 'attention_mask', 'offset_mapping', 'overflow_to_sample_mapping'])

In [28]:
print(inputs['offset_mapping'][1])

[(0, 0), (0, 3), (3, 5), (5, 7), (8, 9), (9, 10), (10, 11), (12, 13), (13, 14), (14, 15), (16, 17), (17, 18), (18, 19), (20, 21), (21, 22), (22, 23), (23, 25), (26, 27), (27, 28), (29, 30), (30, 32), (33, 34), (34, 35), (35, 36), (36, 37), (0, 0), (60, 63), (63, 64), (65, 66), (66, 69), (69, 70), (71, 73), (73, 74), (74, 75), (76, 78), (79, 81), (81, 83), (83, 84), (85, 86), (87, 91), (91, 92), (93, 94), (94, 95), (96, 97), (97, 98), (98, 99), (100, 102), (103, 104), (104, 105), (105, 106), (107, 108), (108, 109), (109, 110), (110, 111), (112, 113), (113, 114), (114, 115), (116, 117), (117, 118), (118, 119), (120, 121), (121, 122), (122, 123), (124, 125), (125, 126), (126, 127), (127, 128), (129, 131), (131, 132), (133, 134), (134, 135), (136, 137), (137, 138), (139, 140), (140, 141), (142, 144), (144, 145), (145, 146), (147, 148), (148, 149), (149, 150), (150, 151), (152, 154), (154, 155), (155, 156), (156, 157), (158, 159), (159, 160), (160, 161), (162, 165), (165, 166), (167, 168), 

In [29]:
inputs = tokenizer(
    raw_datasets["train"][2:6]["question"],
    raw_datasets["train"][2:6]["context"],
    max_length=100,
    truncation="only_second",
    stride=50,
    return_overflowing_tokens=True,
    return_offsets_mapping=True,
)

print(f"The 4 examples gave {len(inputs['input_ids'])} features.")
print(f"Here is where each comes from: {inputs['overflow_to_sample_mapping']}.")

The 4 examples gave 70 features.
Here is where each comes from: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3].


In [30]:
print(len(inputs['input_ids'][3]))
print(len(inputs['input_ids'][7]))
print(len(inputs['input_ids'][11]))
print(len(inputs['input_ids'][18]))
# other are of length 100

100
100
100
100


- nếu câu hỏi yêu cầu một thông tin mà không tồn tại trong ngữ cảnh đã cho (ví dụ: câu trả lời không được đề cập trong đoạn văn), thì mô hình sẽ trả về giá trị (0, 0) để biểu thị rằng không có câu trả lời nào.
- Nếu câu trả lời nằm trong đoạn ngữ cảnh, thì vị trí sẽ được đánh dấu bằng (start_position, end_position):
  + start_position: chỉ số của token (mã số) đầu tiên trong input IDs nơi câu trả lời bắt đầu. Token đầu tiên của câu trả lời sẽ có chỉ số này.
  + end_position: chỉ số của token (mã số) cuối cùng trong input IDs nơi câu trả lời kết thúc. Token cuối cùng của câu trả lời sẽ có chỉ số này.

- Ví dụ minh họa
  + context: "Hà Nội là thủ đô của Việt Nam."
  + Nếu câu hỏi là: "Thủ đô của Việt Nam là gì?"
  + Câu trả lời sẽ là "Hà Nội".
  => Khi đó:
  + Nếu mô hình tìm thấy câu trả lời trong ngữ cảnh, nó sẽ đánh dấu vị trí của từ "Hà Nội" bằng (start_position, end_position), nơi start_position là vị trí của token "Hà" và end_position là vị trí của token "Nội".
  + Nếu câu hỏi là: "Thủ đô của Thái Lan là gì?", và ngữ cảnh không có thông tin về điều đó, thì mô hình sẽ đánh dấu vị trí là (0, 0).

In [31]:
inputs.keys()

dict_keys(['input_ids', 'token_type_ids', 'attention_mask', 'offset_mapping', 'overflow_to_sample_mapping'])

input_ids: Danh sách các mã token cho câu hỏi và ngữ cảnh.
attention_mask: Biến thể để chỉ ra đâu là token thực sự và đâu là padding (nếu có).
overflow_to_sample_mapping: Chỉ ra mối quan hệ giữa các mẫu trong dữ liệu gốc và các mẫu đã được tạo ra qua token hóa.
offset_mapping: Thông tin về vị trí của từng token trong văn bản gốc, giúp xác định các vị trí bắt đầu và kết thúc của câu trả lời trong ngữ cảnh.

In [32]:
answers = raw_datasets["train"][2:6]["answers"]
contexts = raw_datasets["train"][2:6]["context"]  # Giả sử bạn có context ở đây
start_positions = []
end_positions = []

for i, offset in enumerate(inputs["offset_mapping"]):
    sample_idx = inputs["overflow_to_sample_mapping"][i]
    answer = answers[sample_idx]["text"][0]  # Câu trả lời text đầu tiên
    context = contexts[sample_idx]

    # Tìm vị trí start_char và end_char bằng cách tìm vị trí của answer trong context
    start_char = context.find(answer)
    end_char = start_char + len(answer)

    # Nếu không tìm thấy câu trả lời trong context, gán (0, 0)
    if start_char == -1:
        start_positions.append(0)
        end_positions.append(0)
        continue

    sequence_ids = inputs.sequence_ids(i)

    # Tìm vị trí bắt đầu và kết thúc của context trong dãy sequence_ids
    idx = 0
    while sequence_ids[idx] != 1:
        idx += 1
    context_start = idx
    while sequence_ids[idx] == 1:
        idx += 1
    context_end = idx - 1

    # Nếu câu trả lời không hoàn toàn nằm trong context, gán (0, 0)
    if offset[context_start][0] > start_char or offset[context_end][1] < end_char:
        start_positions.append(0)
        end_positions.append(0)
    else:
        # Xác định vị trí bắt đầu và kết thúc của token tương ứng với câu trả lời
        idx = context_start
        while idx <= context_end and offset[idx][0] <= start_char:
            idx += 1
        start_positions.append(idx - 1)

        idx = context_end
        while idx >= context_start and offset[idx][1] >= end_char:
            idx -= 1
        end_positions.append(idx + 1)

start_positions, end_positions


([0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  98,
  79,
  60,
  41,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0],
 [0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  98,
  79,
  60,
  41,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0])

In [33]:
print(inputs.sequence_ids(18))

[None, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, None, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, None]


In [34]:
for i, offset in enumerate(inputs["offset_mapping"]):
  print(i,offset)
  if i==1:
    break

0 [(0, 0), (0, 3), (3, 5), (5, 7), (8, 9), (9, 10), (10, 11), (12, 19), (19, 20), (20, 21), (22, 27), (28, 29), (29, 30), (31, 34), (35, 37), (37, 38), (38, 40), (41, 43), (44, 45), (45, 46), (47, 51), (52, 53), (53, 54), (55, 56), (56, 58), (59, 60), (60, 61), (61, 62), (62, 63), (0, 0), (0, 7), (8, 9), (9, 15), (16, 23), (23, 24), (24, 30), (31, 32), (32, 33), (34, 35), (36, 37), (38, 39), (39, 40), (41, 42), (43, 44), (45, 46), (47, 48), (48, 49), (50, 51), (52, 55), (55, 56), (56, 57), (57, 59), (59, 60), (60, 63), (63, 64), (65, 66), (66, 69), (69, 70), (71, 73), (73, 74), (74, 75), (76, 78), (79, 81), (81, 83), (83, 84), (85, 86), (87, 91), (91, 92), (93, 94), (94, 95), (96, 97), (97, 98), (98, 99), (100, 102), (103, 104), (104, 105), (105, 106), (107, 108), (108, 109), (109, 110), (110, 111), (112, 113), (113, 114), (114, 115), (116, 117), (117, 118), (118, 119), (120, 121), (121, 122), (122, 123), (124, 125), (125, 126), (126, 127), (127, 128), (129, 131), (131, 132), (133, 134

In [35]:
print(inputs['input_ids'][0])

[101, 24896, 1320, 2093, 187, 28659, 1182, 16784, 112, 188, 6405, 191, 9183, 100, 24438, 9183, 15624, 11019, 188, 28227, 3444, 189, 28664, 180, 3031, 183, 9183, 1186, 136, 102, 24041, 144, 22080, 25384, 118, 5007, 113, 120, 171, 178, 176, 21409, 100, 183, 188, 174, 28283, 120, 17775, 118, 162, 11414, 118, 1474, 114, 113, 11850, 1324, 21174, 9183, 1183, 5129, 24438, 4881, 1403, 130, 2358, 114, 181, 9183, 182, 28657, 1204, 11019, 188, 28227, 117, 183, 1324, 28640, 1665, 188, 28227, 117, 183, 1324, 9183, 188, 28641, 1179, 193, 1358, 28642, 1204, 24438, 1358, 248, 1306, 191, 102]


In [36]:
idx = 0
sample_idx = inputs["overflow_to_sample_mapping"][idx]
answer = answers[sample_idx]["text"][0]

start = start_positions[idx]
end = end_positions[idx]
labeled_answer = tokenizer.decode(inputs["input_ids"][idx][start : end + 1])

print(f"Theoretical answer: {answer}, labels give: {labeled_answer}")

Theoretical answer: 2003, labels give: [CLS]


In [37]:
idx = 4
sample_idx = inputs["overflow_to_sample_mapping"][idx]
answer = answers[sample_idx]["text"][0]

decoded_example = tokenizer.decode(inputs["input_ids"][idx])
print(f"Theoretical answer: {answer}")
print(f"labels give: {decoded_example}")

Theoretical answer: 2003
labels give: [CLS] Beyonce rời Destiny's Child và [UNK] thành ca sĩ solo từ khi nào? [SEP]iên người Mỹ. Sinh ra và lớn lên [UNK] Houston, Texas, cô đã biểu diễn trong các cuộc thi ca hát và nhảy múa khác nhau khi [SEP]


In [38]:
max_length = 384
stride = 128

def preprocess_training_examples(examples):
    questions = [q.strip() for q in examples["question"]]
    inputs = tokenizer(
        questions,
        examples["context"],
        max_length=max_length,
        truncation="only_second",
        stride=stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    offset_mapping = inputs.pop("offset_mapping")
    sample_map = inputs.pop("overflow_to_sample_mapping")
    answers = examples["answers"]
    contexts = examples["context"]
    start_positions = []
    end_positions = []

    for i, offset in enumerate(offset_mapping):
        sample_idx = sample_map[i]
        answer_text = answers[sample_idx]["text"][0]  # Lấy câu trả lời đầu tiên
        context = contexts[sample_idx]

        # Tìm start_char và end_char bằng cách tìm vị trí của answer_text trong context
        start_char = context.find(answer_text)
        end_char = start_char + len(answer_text) if start_char != -1 else -1

        sequence_ids = inputs.sequence_ids(i)

        # Tìm vị trí bắt đầu và kết thúc của context trong dãy sequence_ids
        idx = 0
        while sequence_ids[idx] != 1:
            idx += 1
        context_start = idx
        while sequence_ids[idx] == 1:
            idx += 1
        context_end = idx - 1

        # Nếu câu trả lời không nằm trong context hoặc không khớp, gán (0, 0)
        if start_char == -1 or offset[context_start][0] > start_char or offset[context_end][1] < end_char:
            start_positions.append(0)
            end_positions.append(0)
        else:
            # Tìm vị trí bắt đầu và kết thúc của token tương ứng với câu trả lời
            idx = context_start
            while idx <= context_end and offset[idx][0] <= start_char:
                idx += 1
            start_positions.append(idx - 1)

            idx = context_end
            while idx >= context_start and offset[idx][1] >= end_char:
                idx -= 1
            end_positions.append(idx + 1)

    inputs["start_positions"] = start_positions
    inputs["end_positions"] = end_positions
    return inputs


In [39]:
train_dataset = raw_datasets["train"].map(
    preprocess_training_examples,
    batched=True,
    remove_columns=raw_datasets["train"].column_names,
)
len(raw_datasets["train"]), len(train_dataset)

Map:   0%|          | 0/1303 [00:00<?, ? examples/s]

(1303, 2773)

In [40]:
train_dataset

Dataset({
    features: ['input_ids', 'token_type_ids', 'attention_mask', 'start_positions', 'end_positions'],
    num_rows: 2773
})

In [41]:
for i in range(0,88729):
  found = False
  for j in tokenizer.decode(train_dataset[i]['input_ids']).split():
    if j=="[PAD]":
      found = True
  if found==False:
    print(i)
    break

0


In [42]:
tokenizer.decode(train_dataset[249]['input_ids'])

'[CLS] " Independent Women Part I " duy nhất của họ đứng đầu bảng xếp hạng trong bao nhiêu tuần? [SEP] Các thành viên ban nhạc còn lại đã thu âm " Phụ nữ độc lập Phần I ", xuất hiện trên nhạc nền cho bộ phim 2000, Charlie\'s Angels. Nó đã [UNK] thành đĩa đơn xếp hạng tốt nhất của họ, đứng đầu bảng xếp hạng Billboard Hot 100 của Hoa Kỳ trong mười một tuần liên tiếp. Vào đầu năm 2001, trong khi Destiny\'s Child đang hoàn thành album thứ ba của họ, Beyoncé đã giành được vai trò chính trong bộ phim MTV được sản xuất cho truyền hình, Carmen : Hip Hopera, với sự tham gia của nam diễn viên người Mỹ Mekhi Phifer. Lấy bối cảnh [UNK] Philadelphia, bộ phim là một cách giải thích hiện đại về [UNK] opera thế [UNK] 19 Carmen của nhà soạn n [SEP]'

In [43]:
tokenizer.decode(train_dataset[250]['input_ids'])

'[CLS] " Independent Women Part I " duy nhất của họ đứng đầu bảng xếp hạng trong bao nhiêu tuần? [SEP]ò chính trong bộ phim MTV được sản xuất cho truyền hình, Carmen : Hip Hopera, với sự tham gia của nam diễn viên người Mỹ Mekhi Phifer. Lấy bối cảnh [UNK] Philadelphia, bộ phim là một cách giải thích hiện đại về [UNK] opera thế [UNK] 19 Carmen của nhà soạn nhạc người Pháp Georges Bizet. Khi album thứ ba Survivor được phát hành vào tháng 5 2001, Luckett và Roberson đã đệ đơn kiện tuyên bố [UNK] các bài hát nhắm vào họ. Album ra mắt [UNK] vị trí số một trên Billboard 200 của Hoa Kỳ, với doanh số tuần đầu tiên là 663. 000 bản được bán. Album đã sinh ra các bản hit số một khác, " Bootylicious " và ca khúc chủ đề, " Survivor ", sau đó đã mang lạ [SEP]'

In [44]:
raw_datasets['train'][249]['answers']['text'][0]

'Tinh Chất'

In [45]:

print(train_dataset[249]['start_positions'])
print(train_dataset[249]['end_positions'])

0
0


In [46]:
examples = raw_datasets['train'][249]
inputs = tokenizer(examples['question'],
                   examples['context'],
                   max_length=max_length,
                   truncation='only_second',
                   stride=stride,
                   return_overflowing_tokens=True,
                   return_offsets_mapping=True,
                   padding='max_length')
offset_mapping = inputs.pop('offset_mapping')
for i, offset in enumerate(offset_mapping):
  if i ==0:
    my_string = tokenizer.decode(train_dataset[249]['input_ids'][23:27])
    print(my_string)

##u bảng


# 3. Processing the validation dataset

In [47]:
def preprocess_validation_examples(examples):
    questions = [q.strip() for q in examples["question"]]
    inputs = tokenizer(
        questions,
        examples["context"],
        max_length=max_length,
        truncation="only_second",
        stride=stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    sample_map = inputs.pop("overflow_to_sample_mapping")

    for i in range(len(inputs["input_ids"])):
        sample_idx = sample_map[i]

        sequence_ids = inputs.sequence_ids(i)
        offset = inputs["offset_mapping"][i]
        inputs["offset_mapping"][i] = [
            o if sequence_ids[k] == 1 else None for k, o in enumerate(offset)
        ]

    return inputs


In [48]:
validation_dataset = raw_datasets["validation"].map(
    preprocess_validation_examples,
    batched=True,
    remove_columns=raw_datasets["validation"].column_names,
)
len(raw_datasets["validation"]), len(validation_dataset)

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

(1000, 1637)

In [49]:
validation_dataset

Dataset({
    features: ['input_ids', 'token_type_ids', 'attention_mask', 'offset_mapping'],
    num_rows: 1637
})

In [50]:
# validation_dataset[0]['example_id']

# 4. Fine tuning

In [51]:

from huggingface_hub import notebook_login

notebook_login()

In [52]:
from transformers import TrainingArguments

args = TrainingArguments(
    "bert-finetuned-squad",
    evaluation_strategy="no",
    save_strategy="epoch",
    learning_rate=2e-5,
    num_train_epochs=20,
    weight_decay=0.01,
    push_to_hub=True,
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [53]:

from transformers import AutoModelForQuestionAnswering
model = AutoModelForQuestionAnswering.from_pretrained(model_checkpoint)

model.safetensors:   0%|          | 0.00/436M [00:00<?, ?B/s]

Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [54]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_dataset,
    eval_dataset=validation_dataset,
    tokenizer=tokenizer,
)
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Step,Training Loss
500,1.346100
1000,0.941800
1500,0.644000
2000,0.462500
2500,0.316100
3000,0.227300
3500,0.182900
4000,0.109600
4500,0.088800
5000,0.068100


TrainOutput(global_step=6940, training_loss=0.3245883435955652, metrics={'train_runtime': 4321.2135, 'train_samples_per_second': 12.834, 'train_steps_per_second': 1.606, 'total_flos': 1.086863859643392e+16, 'train_loss': 0.3245883435955652, 'epoch': 20.0})

In [55]:
trainer.push_to_hub(commit_message="Training complete")


events.out.tfevents.1730430925.70e207f0e77f.599.0:   0%|          | 0.00/8.01k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/qnt13/bert-finetuned-squad/commit/1c6ef3634d666689acd9f14baa6de35568154af7', commit_message='Training complete', commit_description='', oid='1c6ef3634d666689acd9f14baa6de35568154af7', pr_url=None, pr_revision=None, pr_num=None)

In [56]:
from transformers import pipeline

# Replace this with your own checkpoint
model_checkpoint = "neural-net-rahul/bert-finetuned-squad"
question_answerer = pipeline("question-answering", model=model_checkpoint)

config.json:   0%|          | 0.00/671 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/431M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.19k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/669k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


In [60]:
context = """
Transformers được hỗ trợ bởi ba thư viện học sâu phổ biến nhất — Jax, PyTorch và TensorFlow — với sự tích hợp liền mạch
giữa chúng. Thật dễ dàng để đào tạo các mô hình của bạn bằng một thư viện trước khi tải chúng để suy luận bằng thư viện còn lại.
"""
question = "Những thư viện học sâu nào hỗ trợ Transformers?"
question_answerer(question=question, context=context)['answer']

'Jax'

In [61]:
context = """
Cây xương rồng có rất nhiều loại khác nhau, với nhiều hình dạng và kích thước đa dạng. Một số loại cây xương rồng có thể phát triển rất lớn,
trong khi những loại khác có kích thước nhỏ và mọc thành từng cụm.
Đặc điểm chung của các loại cây xương rồng là khả năng sống sót trong môi trường khắc nghiệt và giữ nước rất tốt.
"""
question = "Đặc điểm chung của các loại cây xương rồng là gì?"
question_answerer(question=question, context=context)['answer']


'khắc nghiệt'

In [65]:
import shutil
from google.colab import files

shutil.make_archive("/content/bert-finetuned-squad", 'zip', "/content/bert-finetuned-squad")

files.download("/content/bert-finetuned-squad.zip")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>